In [1]:
import numpy as np

In [3]:
prob_n=(0.2,0.31,0.19,0.3)

prob_n = np.asarray(prob_n)
csprob_n = np.cumsum(prob_n)

In [4]:
csprob_n

array([0.2 , 0.51, 0.7 , 1.  ])

In [5]:
(csprob_n > np_random.rand()).argmax()

NameError: name 'np_random' is not defined

In [6]:
mydict= {}
mydict[('po', 'lo')]= 5

In [8]:
mydict

{('po', 'lo'): 5}

In [9]:
mydict['po', 'lo']

5

In [4]:
po = np.array((1,2,3,6,5))
po.argmax()

3

In [5]:
if 3 is not po.argmax():
    print('true')

true


In [6]:
do = np.array((2,6,3,5,2))
po-do

array([-1, -4,  0,  1,  3])

In [7]:
maxdiff = float('inf')


In [8]:
maxdiff

inf

In [9]:
maxdiff>100000000000000000

True

In [11]:
do[3]

5

In [23]:
print(po, do)
po = np.array((1,2,3,6,5))
do = np.array((2,6,100,5,2))
max(do, 0)

[1 2 3 6 5] [  2   6 100   5   2]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [21]:
nA =4
action_value =[0 for a in range(nA)]

In [22]:
action_value

[0, 0, 0, 0]

In [16]:
max(abs(po-do))

97

In [ ]:
### MDP Value Iteration and Policy Iteration

import numpy as np
import gym
import time
from itertools import product
from lake_envs import *

np.set_printoptions(precision=3)

"""
For policy_evaluation, policy_improvement, policy_iteration and value_iteration,
the parameters P, nS, nA, gamma are defined as follows:
    P: nested dictionary
        From gym.core.Environment
        For each pair of states in [1, nS] and actions in [1, nA], P[state][action] is a
        tuple of the form (probability, nextstate, reward, terminal) where
            - probability: float
                the probability of transitioning from "state" to "nextstate" with "action"
            - nextstate: int
                denotes the state we transition to (in range [0, nS - 1])
            - reward: int
                either 0 or 1, the reward for transitioning from "state" to
                "nextstate" with "action"
            - terminal: bool
              True when "nextstate" is a terminal state (hole or goal), False otherwise
    nS: int
        number of states in the environment
    nA: int
        number of actions in the environment
    gamma: float
        Discount factor. Number in range [0, 1)
"""

def policy_evaluation(P, nS, nA, policy, gamma=0.9, tol=1e-3):
    """Evaluate the value function from a given policy.
    Parameters
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    policy: np.array[nS]
        The policy to evaluate. Maps states to actions.
    tol: float
        Terminate policy evaluation when
            max |value_function(s) - prev_value_function(s)| < tol
    Returns
    -------
    value_function: np.ndarray[nS]
        The value function of the given policy, where value_function[s] is
        the value of state s
    """
    value_function = np.zeros(nS)

    max_diff = float('inf')

    while max_diff > tol:
        next_value_function = value_function.copy()
        max_diff = -float('inf')
        
        for state in range(nS):
            policy_action = policy[state]
            expected_return = 0
            for  probability, nextstate, reward, terminal in P[state][policy_action]:
                expected_return += probability * (reward + gamma * value_function[nextstate])

            next_value_function[state] = expected_return

            max_diff = max(max_diff, abs(value_function[state] - next_value_function[state]))

        value_function = next_value_function

    return value_function


def policy_improvement(P, nS, nA, value_from_policy, policy, gamma=0.9):
    """Given the value function from policy improve the policy.
    Parameters
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    value_from_policy: np.ndarray
        The value calculated from the policy
    policy: np.array
        The previous policy.
    Returns
    -------
    new_policy: np.ndarray[nS]
        An array of integers. Each integer is the optimal action to take
        in that state according to the environment dynamics and the
        given value function.
    """

    new_policy = np.zeros(nS, dtype='int')

    for state in range(nS):
        max_val = argmax = -float('inf')

        for action in range(nA):
            result = 0
            for prob, nextstate, reward, _ in P[state][action]:
                result += prob * (reward + gamma * value_from_policy[nextstate])

            if result > max_val:
                max_val, argmax = result, action

        new_policy[state] = argmax

    return new_policy


def policy_iteration(P, nS, nA, gamma=0.9, tol=10e-3):
    """Runs policy iteration.
    You should call the policy_evaluation() and policy_improvement() methods to
    implement this method.
    Parameters
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    tol: float
        tol parameter used in policy_evaluation()
    Returns:
    ----------
    value_function: np.ndarray[nS]
    policy: np.ndarray[nS]
    """
    policy = np.zeros(nS)
    value_function = np.zeros(nS)
    maxdiff = float('inf')
    num_iters = 0

    while maxdiff > tol or num_iters < 3:
        next_value_function = policy_evaluation(P, nS, nA, policy, gamma, tol)
        policy = policy_improvement(P, nS, nA, next_value_function, policy, gamma)
        maxdiff = max(abs(next_value_function - value_function))
        value_function = next_value_function
        num_iters += 1

    print(value_function)
    print(f'policy iteration took {num_iters} to converge')
    return value_function, policy

def value_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
    """
    Learn value function and policy by using value iteration method for a given
    gamma and environment.
    Parameters:
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    tol: float
        Terminate value iteration when
            max |value_function(s) - prev_value_function(s)| < tol
    Returns:
    ----------
    value_function: np.ndarray[nS]
    policy: np.ndarray[nS]
    """

    value_function = np.zeros(nS)
    policy = np.zeros(nS, dtype=int)
    
    maxdiff = float('inf')
    num_iters = 0 
    while maxdiff > tol:
        num_iters += 1
        next_value_function = value_function.copy()
        maxdiff = -float('inf')

        for state in range(nS):
            best_return = -float('inf')

            for action in range(nA):
                exp_return = 0
                for prob, nextstate, reward, _ in P[state][action]:
                    exp_return += prob * (reward + gamma * value_function[nextstate])

                best_return = max(best_return, exp_return)

            next_value_function[state] = best_return
            maxdiff = max(maxdiff, abs(next_value_function[state] - value_function[state]))

        value_function = next_value_function

    print(f'value iteration took {num_iters} iterations to converge')
    return value_function, policy_improvement(P, nS, nA, value_function, policy, gamma)

def render_single(env, policy, max_steps=100):
  """
    This function does not need to be modified
    Renders policy once on environment. Watch your agent play!
    Parameters
    ----------
    env: gym.core.Environment
      Environment to play on. Must have nS, nA, and P as
      attributes.
    Policy: np.array of shape [env.nS]
      The action to take at a given state
  """

  episode_reward = 0
  ob = env.reset()
  for t in range(max_steps):
    env.render()
    time.sleep(0.25)
    a = policy[ob]
    ob, rew, done, _ = env.step(a)
    episode_reward += rew
    if done:
      break
  env.render();
  if not done:
    print("The agent didn't reach a terminal state in {} steps.".format(max_steps))
  else:
      print("Episode reward: %f" % episode_reward)


# Edit below to run policy and value iteration on different environments and
# visualize the resulting policies in action!
# You may change the parameters in the functions below
if __name__ == "__main__":

    # comment/uncomment these lines to switch between deterministic/stochastic environments
    env = gym.make("Deterministic-4x4-FrozenLake-v0")
    #env = gym.make("Stochastic-4x4-FrozenLake-v0")

    print("\n" + "-"*25 + "\nBeginning Policy Iteration\n" + "-"*25)

    V_pi, p_pi = policy_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)
    render_single(env, p_pi, 100)

    #print("\n" + "-"*25 + "\nBeginning Value Iteration\n" + "-"*25)

    #V_vi, p_vi = value_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)
    #render_single(env, p_vi, 100)
